<a href="https://colab.research.google.com/github/abhy-kumar/wind-potential/blob/main/osw_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas geopandas folium numpy shapely branca

In [2]:
import requests
import pandas as pd
import folium
import numpy as np
from branca.colormap import LinearColormap
from datetime import datetime, timedelta
import json

def fetch_gfs_wind_data(lat, lon):
    """
    Fetch wind data from NOAA GFS API
    Returns U and V wind components at 100m height
    """
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "YOUR_API_KEY"  # You'll need to sign up for a free API key at OpenWeatherMap

    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data['wind']['speed']
        return None
    except Exception as e:
        print(f"Error fetching data for lat: {lat}, lon: {lon}")
        print(f"Error details: {str(e)}")
        return None

def calculate_wind_power_density(wind_speed):
    """
    Calculate wind power density in W/m²
    Using standard air density at sea level (1.225 kg/m³)
    """
    if wind_speed is None:
        return None
    air_density = 1.225  # kg/m³
    return 0.5 * air_density * (wind_speed ** 3)

def generate_sample_wind_data():
    """
    Generate sample wind data based on typical wind patterns along Indian coast
    This is used when API access is not available
    """
    # Define detailed coastal regions with their characteristics
    coastal_regions = {
        'Gujarat': {
            'start': [(23.3, 68.5)],  # Kutch
            'end': [(20.3, 72.8)],    # Daman
            'base_speed': 8.5,         # Higher wind speeds in Gujarat
            'seasonal_variation': 2.0
        },
        'Maharashtra': {
            'start': [(20.3, 72.8)],
            'end': [(15.7, 73.8)],
            'base_speed': 7.5,
            'seasonal_variation': 1.8
        },
        'Goa': {
            'start': [(15.7, 73.8)],
            'end': [(15.0, 74.0)],
            'base_speed': 7.0,
            'seasonal_variation': 1.5
        },
        'Karnataka': {
            'start': [(15.0, 74.0)],
            'end': [(12.4, 74.9)],
            'base_speed': 6.8,
            'seasonal_variation': 1.5
        },
        'Kerala': {
            'start': [(12.4, 74.9)],
            'end': [(8.1, 77.2)],
            'base_speed': 6.5,
            'seasonal_variation': 2.0
        },
        'Tamil Nadu': {
            'start': [(8.1, 77.2)],
            'end': [(13.5, 80.3)],
            'base_speed': 7.0,
            'seasonal_variation': 2.5  # Higher variation due to monsoons
        },
        'Andhra Pradesh': {
            'start': [(13.5, 80.3)],
            'end': [(19.1, 84.8)],
            'base_speed': 7.2,
            'seasonal_variation': 2.0
        },
        'Odisha': {
            'start': [(19.1, 84.8)],
            'end': [(21.6, 87.5)],
            'base_speed': 7.0,
            'seasonal_variation': 1.8
        },
        'West Bengal': {
            'start': [(21.6, 87.5)],
            'end': [(21.9, 89.0)],    # Sundarbans
            'base_speed': 6.8,
            'seasonal_variation': 1.5
        }
    }

    data = []

    # Generate detailed points along the coastline
    for region, info in coastal_regions.items():
        for start_point, end_point in zip(info['start'], info['end']):
            # Calculate the number of points based on the greater distance
            lat_diff = abs(end_point[0] - start_point[0])
            lon_diff = abs(end_point[1] - start_point[1])
            num_points = int(max(lat_diff, lon_diff) * 10) + 1  # 0.1 degree intervals

            # Create points along the coast
            lats = np.linspace(start_point[0], end_point[0], num_points)
            lons = np.linspace(start_point[1], end_point[1], num_points)

            # Generate points following the coastline
            for lat, lon in zip(lats, lons):

                # Generate realistic wind speeds based on location and regional characteristics
                base_speed = info['base_speed']

                # Add seasonal and local variations
                seasonal_factor = np.sin(2 * np.pi * (datetime.now().month / 12)) * info['seasonal_variation']
                local_variation = np.random.normal(0, 0.5)  # Local geographic variations

                # Calculate final wind speed
                wind_speed = base_speed + seasonal_factor + local_variation

                # Add offshore boost (wind speeds typically increase offshore)
                offshore_boost = np.random.uniform(0.5, 1.5)
                wind_speed += offshore_boost

                # Keep within realistic bounds
                wind_speed = max(4, min(14, wind_speed))

                # Calculate power density
                power_density = calculate_wind_power_density(wind_speed)

                # Add point to dataset
                data.append({
                    'latitude': lat,
                    'longitude': lon,
                    'wind_speed': wind_speed,
                    'power_density': power_density,
                    'region': region
                })

                # Add offshore points (20km and 40km from coast)
                for distance in [0.2, 0.4]:  # Approximate degree offsets for 20km and 40km
                    if region in ['Gujarat', 'Maharashtra', 'Goa', 'Karnataka', 'Kerala']:
                        # West coast - add points to the west
                        offshore_lon = lon - distance
                    else:
                        # East coast - add points to the east
                        offshore_lon = lon + distance

                    # Increase wind speed for offshore points
                    offshore_wind = wind_speed + distance * 2 + np.random.normal(0, 0.3)
                    offshore_wind = max(5, min(15, offshore_wind))

                    data.append({
                        'latitude': lat,
                        'longitude': offshore_lon,
                        'wind_speed': offshore_wind,
                        'power_density': calculate_wind_power_density(offshore_wind),
                        'region': f"{region} (Offshore)"
                    })

    return pd.DataFrame(data)

def create_wind_potential_map():
    print("Generating wind data...")

    # Use sample data since we can't access the API without a key
    df = generate_sample_wind_data()

    print(f"Creating map with {len(df)} data points...")

    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Average Wind Speed: {df['wind_speed'].mean():.2f} m/s")
    print(f"Average Power Density: {df['power_density'].mean():.2f} W/m²")

    # Create map centered on India
    m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

    # Create color map
    colormap = LinearColormap(
        colors=['yellow', 'orange', 'red'],
        vmin=df['power_density'].min(),
        vmax=df['power_density'].max(),
        caption='Wind Power Density (W/m²)'
    )

    # Add points to map with region information
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=10,
            color=None,
            fill=True,
            fill_color=colormap(row['power_density']),
            fill_opacity=0.7,
            popup=f"Region: {row['region']}<br>"
                  f"Wind Speed: {row['wind_speed']:.1f} m/s<br>"
                  f"Power Density: {row['power_density']:.0f} W/m²"
        ).add_to(m)

    # Add colormap to map
    colormap.add_to(m)

    # Add title
    title_html = '''
        <div style="position: fixed;
                    top: 10px; left: 50px; width: 300px; height: 30px;
                    z-index:9999; font-size:16px; background-color: white;
                    border-radius: 5px; padding: 10px">
            <b>Indian Coastal Wind Power Potential</b>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Save map
    output_file = 'india_offshore_wind_potential.html'
    m.save(output_file)
    print(f"\nMap saved as '{output_file}'")

    # Save data to CSV for reference
    df.to_csv('wind_potential_data.csv', index=False)
    print("Data saved to 'wind_potential_data.csv'")

if __name__ == "__main__":
    create_wind_potential_map()

Generating wind data...
Creating map with 969 data points...

Summary Statistics:
Average Wind Speed: 9.62 m/s
Average Power Density: 560.41 W/m²

Map saved as 'india_offshore_wind_potential.html'
Data saved to 'wind_potential_data.csv'


In [6]:
!pip install requests pandas folium numpy pygrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 50.2 MB/s eta 0:00:00


In [7]:
import requests
import pandas as pd
import folium
import numpy as np
from branca.colormap import LinearColormap
from folium.plugins import HeatMap
from datetime import datetime, timedelta

def fetch_real_wind_data():
    """Fetch real wind data from NOAA GFS (Global Forecast System)"""
    # Get current time in UTC
    now = datetime.utcnow()
    base_time = now - timedelta(hours=now.hour % 6)
    time_str = base_time.strftime("%Y%m%d%H")

    # Define area of interest around Indian coastline
    bbox = [5.0, 65.0, 25.0, 95.0]  # lat_min, lon_min, lat_max, lon_max
    resolution = 0.25  # degrees between points

    # GFS API endpoint parameters
    url = "https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25.pl"
    params = {
        'file': f'gfs.t{base_time.hour:02d}z.pgrb2.0p25.f000',
        'var_UGRD': 'on',
        'var_VGRD': 'on',
        'lev_100_m_above_ground': 'on',
        'subregion': '',
        'leftlon': bbox[1],
        'rightlon': bbox[3],
        'toplat': bbox[2],
        'bottomlat': bbox[0],
        'dir': f'/gfs.{time_str[:-2]}/{base_time.hour:02d}/atmos'
    }

    try:
        response = requests.get(url, params=params, timeout=30)
        if response.status_code == 200:
            return parse_gfs_data(response.content)
        return None
    except Exception as e:
        print(f"Error fetching GFS data: {str(e)}")
        return None

def parse_gfs_data(content):
    """Parse binary GFS data into usable wind speeds"""
    # This requires the pygrib package - install with: pip install pygrib
    try:
        import pygrib
    except ImportError:
        print("pygrib package required for GFS data parsing")
        return None

    grbs = pygrib.open_from_memory(content)
    u_wind = grbs.select(name='U component of wind')[0]
    v_wind = grbs.select(name='V component of wind')[0]

    lats, lons = u_wind.latlons()
    u_data = u_wind.values
    v_data = v_wind.values
    wind_speeds = np.sqrt(u_data**2 + v_data**2)

    data = []
    for i in range(wind_speeds.shape[0]):
        for j in range(wind_speeds.shape[1]):
            if 65 <= lons[i,j] <= 95 and 5 <= lats[i,j] <= 25:  # Filter for India
                data.append({
                    'latitude': lats[i,j],
                    'longitude': lons[i,j],
                    'wind_speed': wind_speeds[i,j],
                    'power_density': 0.5 * 1.225 * (wind_speeds[i,j] ** 3)
                })

    return pd.DataFrame(data)

def create_wind_map():
    # Try to get real data first
    df = fetch_real_wind_data()

    # Fallback to sample data if real data fails
    if df is None or df.empty:
        print("Using sample data as fallback")
        df = generate_fallback_data()

    # Clean data
    df = df.dropna()
    df = df[(df['power_density'] > 0) & (df['power_density'] < 2000)]

    # Create map centered on India
    m = folium.Map(location=[20.5937, 78.9629], zoom_start=5,
                  tiles='CartoDB positron', control_scale=True)

    # Create heatmap layer
    heat_data = [[row['latitude'], row['longitude'], row['power_density']]
                 for _, row in df.iterrows()]

    HeatMap(
        heat_data,
        radius=15,
        blur=25,
        gradient={
            '0.1': '#2c7bb6',
            '0.4': '#abd9e9',
            '0.6': '#ffffbf',
            '0.8': '#fdae61',
            '1.0': '#d7191c'
        },
        min_opacity=0.5,
        max_zoom=12,
    ).add_to(m)

    # Add colormap legend
    colormap = LinearColormap(
        colors=['#2c7bb6', '#abd9e9', '#ffffbf', '#fdae61', '#d7191c'],
        vmin=df['power_density'].min(),
        vmax=df['power_density'].max(),
        caption='Wind Power Density (W/m²)'
    ).add_to(m)

    # Add title
    title_html = '''
        <div style="position: fixed; top: 10px; left: 50px; z-index:9999;
                    background: rgba(255,255,255,0.8); padding: 10px;
                    border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.2)">
            <h4 style="margin:0">Indian Offshore Wind Power Potential</h4>
            <p style="margin:3px 0; font-size:0.9em">{date}</p>
            <p style="margin:3px 0; font-size:0.8em; color:#666">
                Data source: {source}
            </p>
        </div>
    '''.format(
        date=datetime.now().strftime("%Y-%m-%d %H:%M UTC"),
        source="NOAA GFS" if 'fallback' not in df.columns else "Sample Data"
    )
    m.get_root().html.add_child(folium.Element(title_html))

    m.save('wind_power_map.html')
    print("Map saved successfully!")

def generate_fallback_data():
    """Generate realistic sample data for India"""
    np.random.seed(42)
    lats = np.concatenate([
        np.linspace(8.0, 23.0, 50),
        np.linspace(8.0, 23.0, 50)
    ])
    lons = np.concatenate([
        np.linspace(68.0, 73.0, 50),  # West coast
        np.linspace(77.0, 82.0, 50)   # East coast
    ])

    data = []
    for lat, lon in zip(lats, lons):
        # Base speed with coastal variation
        base_speed = 7 + (lat * 0.2) + np.random.uniform(-1, 1)
        # Add offshore boost
        if lon > 75:  # East coast
            base_speed += np.abs(lon - 77) * 0.3
        else:         # West coast
            base_speed += np.abs(73 - lon) * 0.3

        wind_speed = max(4, min(14, base_speed + np.random.normal(0, 1)))
        power_density = 0.5 * 1.225 * (wind_speed ** 3)

        data.append({
            'latitude': lat,
            'longitude': lon,
            'wind_speed': wind_speed,
            'power_density': power_density,
            'fallback': True
        })

    return pd.DataFrame(data)

if __name__ == "__main__":
    create_wind_map()

Using sample data as fallback
Map saved successfully!
